In [74]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 20: Jurassic Jigsaw ---</h2><p>The high-speed train leaves the forest and quickly carries you south. You can even see a desert in the distance! Since you have some spare time, you <span title="Just in case. Maybe they missed something.">might as well</span> see if there was anything interesting in the image the Mythical Information Bureau satellite captured.</p>
<p>After decoding the satellite messages, you discover that the data actually contains many small images created by the satellite's <em>camera array</em>. The camera array consists of many cameras; rather than produce a single square image, they produce many smaller square image <em>tiles</em> that need to be <em>reassembled back into a single image</em>.</p>
<p>Each camera in the camera array returns a single monochrome <em>image tile</em> with a random unique <em>ID number</em>.  The tiles (your puzzle input) arrived in a random order.</p>
<p>Worse yet, the camera array appears to be malfunctioning: each image tile has been <em>rotated and flipped to a random orientation</em>. Your first task is to reassemble the original image by orienting the tiles so they fit together.</p>
<p>To show how the tiles should be reassembled, each tile's image data includes a border that should line up exactly with its adjacent tiles. All tiles have this border, and the border lines up exactly when the tiles are both oriented correctly. Tiles at the edge of the image also have this border, but the outermost edges won't line up with any other tiles.</p>
<p>For example, suppose you have the following nine tiles:</p>
<pre><code>Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
</code></pre>

<p>By rotating, flipping, and rearranging them, you can find a square arrangement that causes all adjacent borders to line up:</p>
<pre><code>#...##.#.. ..###..### #.#.#####.
..#.#..#.# ###...#.#. .#..######
.###....#. ..#....#.. ..#.......
###.##.##. .#.#.#..## ######....
.###.##### ##...#.### ####.#..#.
.##.#....# ##.##.###. .#...#.##.
#...###### ####.#...# #.#####.##
.....#..## #...##..#. ..#.###...
#.####...# ##..#..... ..#.......
#.##...##. ..##.#..#. ..#.###...

#.##...##. ..##.#..#. ..#.###...
##..#.##.. ..#..###.# ##.##....#
##.####... .#.####.#. ..#.###..#
####.#.#.. ...#.##### ###.#..###
.#.####... ...##..##. .######.##
.##..##.#. ....#...## #.#.#.#...
....#..#.# #.#.#.##.# #.###.###.
..#.#..... .#.##.#..# #.###.##..
####.#.... .#..#.##.. .######...
...#.#.#.# ###.##.#.. .##...####

...#.#.#.# ###.##.#.. .##...####
..#.#.###. ..##.##.## #..#.##..#
..####.### ##.#...##. .#.#..#.##
#..#.#..#. ...#.#.#.. .####.###.
.#..####.# #..#.#.#.# ####.###..
.#####..## #####...#. .##....##.
##.##..#.. ..#...#... .####...#.
#.#.###... .##..##... .####.##.#
#...###... ..##...#.. ...#..####
..#.#....# ##.#.#.... ...##.....
</code></pre>

<p>For reference, the IDs of the above tiles are:</p>
<pre><code><em>1951</em>    2311    <em>3079</em>
2729    1427    2473
<em>2971</em>    1489    <em>1171</em>
</code></pre>
<p>To check that you've assembled the image correctly, multiply the IDs of the four corner tiles together. If you do this with the assembled tiles from the example above, you get <code>1951 * 3079 * 2971 * 1171</code> = <em><code>20899048083289</code></em>.</p>
<p>Assemble the tiles into an image. <em>What do you get if you multiply together the IDs of the four corner tiles?</em></p>
</article>


In [75]:
example = """
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
"""

In [76]:
from math import prod
from pprint import pprint
from tabulate import tabulate


def assemble(s: str) -> int:
    tiles = [t.splitlines() for t in re.split(r"\n\s*\n", s.strip())]

    side_to_tiles = defaultdict(list)

    for tile in tiles:
        nr, *tile = tile
        nr = int(re.sub(r"\D+", "", nr))

        top = tile[0]
        left = "".join(l[-1] for l in tile)
        bottom = tile[-1]
        right = "".join(l[0] for l in tile)

        side_to_tiles[top].append(nr)
        side_to_tiles[left].append(nr)
        side_to_tiles[bottom].append(nr)
        side_to_tiles[right].append(nr)

        side_to_tiles[top[::-1]].append(nr)
        side_to_tiles[left[::-1]].append(nr)
        side_to_tiles[bottom[::-1]].append(nr)
        side_to_tiles[right[::-1]].append(nr)

    possible_pairs = {tuple(sorted(p)) for p in side_to_tiles.values() if len(p) == 2}

    tile_neigbors = defaultdict(list)
    for t1, t2 in possible_pairs:
        tile_neigbors[t1].append(t2)
        tile_neigbors[t2].append(t1)

    return prod(k for k, v in tile_neigbors.items() if len(v) == 2)


assert assemble(example) == 20899048083289

In [77]:
with open("../input/day20.txt") as f:
    puzzle = f.read()

print(f"Part I: {assemble(puzzle)}")

Part I: 17148689442341


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>17148689442341</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, you're ready to <em>check the image for sea monsters</em>.</p>
<p>The borders of each tile are not part of the actual image; start by removing them.</p>
<p>In the example above, the tiles become:</p>
<pre><code>.#.#..#. ##...#.# #..#####
###....# .#....#. .#......
##.##.## #.#.#..# #####...
###.#### #...#.## ###.#..#
##.#.... #.##.### #...#.##
...##### ###.#... .#####.#
....#..# ...##..# .#.###..
.####... #..#.... .#......

#..#.##. .#..###. #.##....
#.####.. #.####.# .#.###..
###.#.#. ..#.#### ##.#..##
#.####.. ..##..## ######.#
##..##.# ...#...# .#.#.#..
...#..#. .#.#.##. .###.###
.#.#.... #.##.#.. .###.##.
###.#... #..#.##. ######..

.#.#.### .##.##.# ..#.##..
.####.## #.#...## #.#..#.#
..#.#..# ..#.#.#. ####.###
#..####. ..#.#.#. ###.###.
#####..# ####...# ##....##
#.##..#. .#...#.. ####...#
.#.###.. ##..##.. ####.##.
...###.. .##...#. ..#..###
</code></pre>

<p>Remove the gaps to form the actual image:</p>
<pre><code>.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###
</code></pre>
<p>Now, you're ready to search for sea monsters! Because your image is monochrome, a sea monster will look like this:</p>
<pre><code>                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
</code></pre>
<p>When looking for this pattern in the image, <em>the spaces can be anything</em>; only the <code>#</code> need to match. Also, you might need to rotate or flip your image before it's oriented correctly to find sea monsters. In the above image, <em>after flipping and rotating it</em> to the appropriate orientation, there are <em>two</em> sea monsters (marked with <code><em>O</em></code>):</p>
<pre><code>.####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.<em>O</em>#..
#.<em>O</em>.##.<em>O</em><em>O</em>#.#.<em>O</em><em>O</em>.##.<em>O</em><em>O</em><em>O</em>##
..#<em>O</em>.#<em>O</em>#.<em>O</em>##<em>O</em>..<em>O</em>.#<em>O</em>##.##
...#.#..##.##...#..#..##
#.##.#..#.#..#..##.#.#..
.###.##.....#...###.#...
#.####.#.#....##.#..#.#.
##...#..#....#..#...####
..#.##...###..#.#####..#
....#.##.#.#####....#...
..##.##.###.....#.##..#.
#...#...###..####....##.
.#.##...#.##.#.#.###...#
#.###.#..####...##..#...
#.###...#.##...#.##<em>O</em>###.
.<em>O</em>##.#<em>O</em><em>O</em>.###<em>O</em><em>O</em>##..<em>O</em><em>O</em><em>O</em>##.
..<em>O</em>#.<em>O</em>..<em>O</em>..<em>O</em>.#<em>O</em>##<em>O</em>##.###
#.#..##.########..#..##.
#.#####..#.#...##..#....
#....##..#.#########..##
#...#.....#..##...###.##
#..###....##.#...##.##.#
</code></pre>
<p>Determine how rough the waters are in the sea monsters' habitat by counting the number of <code>#</code> that are <em>not</em> part of a sea monster. In the above example, the habitat's water roughness is <em><code>273</code></em>.</p>
<p><em>How many <code>#</code> are not part of a sea monster?</em></p>
</article>

</main>


In [78]:
from math import isqrt

from more_itertools import first
from pyparsing import deque


class Tile:
    def __init__(self, s) -> None:
        nr, *tile = s
        self.nr = int(re.sub(r"\D+", "", nr))
        self.tile = tile
        self._borders()

    def _borders(self):
        self.top = self.tile[0]
        self.left = "".join(l[0] for l in self.tile)
        self.bottom = self.tile[-1]
        self.right = "".join(l[-1] for l in self.tile)

    def rotate_90_deg(self) -> Tile:
        self.tile = ["".join(reversed(col)) for col in zip(*self.tile)]
        self._borders()
        return self

    def flip_horizontal(self) -> Tile:
        self.tile = [row[::-1] for row in self.tile]
        self._borders()
        return self

    def flip_vertictal(self) -> Tile:
        self.tile = [row for row in reversed(self.tile)]
        self._borders()
        return self

    def get_side(self, other: Tile) -> str | None:
        for i in range(4):
            if i > 0:
                other.rotate_90_deg()
            if self.top == other.bottom:
                return "TOP"
            if self.bottom == other.top:
                return "BOTTOM"
            if self.right == other.left:
                return "RIGHT"
            if self.left == other.right:
                return "LEFT"
        return None

    def neighbors_where(self, other: Tile) -> str:
        side = self.get_side(other)
        if side is not None:
            return side

        other.rotate_90_deg()
        other.flip_vertictal()

        side = self.get_side(other)
        if side is not None:
            return side

    def __eq__(self, other: Tile) -> bool:
        return self.nr == other.nr and self.tile == other.tile

    def __hash__(self) -> int:
        return self.nr

    def __repr__(self) -> str:
        return "\n".join((f"Tile {self.nr}:", "\n".join(row for row in self.tile)))


class Image:
    opsosite_side = {"TOP": "BOTTOM", "BOTTOM": "TOP", "RIGHT": "LEFT", "LEFT": "RIGHT"}

    def __init__(self, s: str) -> None:
        tiles_map, tile_neigbors = self._get_neigbors(s)
        self.grid = self._create_grid(tiles_map, tile_neigbors)

    def rotate_90_deg(self) -> Image:
        self.grid = [
            [t.rotate_90_deg() for t in reversed(col)] for col in zip(*self.grid)
        ]
        return self

    def flip_horizontal(self) -> Image:
        self.grid = [[t.flip_horizontal() for t in reversed(row)] for row in self.grid]
        return self

    def flip_vertictal(self) -> Image:
        self.grid = [[t.flip_vertictal() for t in row] for row in reversed(self.grid)]
        return self

    def to_str(self) -> str:
        s = self.make_image()
        return "\n".join("".join(r) for r in s)

    def make_image(self) -> list[list[list[list[chr]]]]:
        n = len(self.grid[0][0].tile)
        s = []
        for row in self.grid:
            for rr in range(1, n - 1):
                s.append([col.tile[rr][cc] for col in row for cc in range(1, n - 1)])
        return s

    def count_pattern_match(self, pattern: list[list[chr]]) -> int:
        image = self.make_image()
        p_rows, p_cols = len(pattern), len(pattern[0])
        i_rows, i_cols = len(image), len(image[0])
        return sum(
            1
            for i_row, i_col in product(
                range(i_rows - p_rows + 1), range(i_cols - p_cols + 1)
            )
            if self.pattern_fits(pattern, image, p_rows, p_cols, i_row, i_col)
        )

    def pattern_fits(self, pattern, image, p_rows, p_cols, i_row, i_col):
        return not any(
            pattern[p_row][p_col] == "#" and image[i_row + p_row][i_col + p_col] != "#"
            for p_row, p_col in product(range(p_rows), range(p_cols))
        )

    def to_str_with_gaps_and_borders(self) -> str:
        n = len(self.grid[0][0].tile)
        s = []
        for row in self.grid:
            for rr in range(n):
                s.append(" ".join(col.tile[rr] for col in row))
            s.append("")
        return "\n".join(s)

    def _create_grid(self, tiles_map, tile_neigbors):
        grid_map = defaultdict(dict)
        queue = deque([first(tile_neigbors.items())])
        seen = set()

        while queue:
            anchor, neighbors = queue.popleft()
            if anchor in seen:
                continue

            seen.add(anchor)
            anchor = tiles_map[anchor]

            grid_map[anchor] = {}
            for neighbor in neighbors:
                neighbor = tiles_map[neighbor]
                side = anchor.neighbors_where(neighbor)

                if side in grid_map[anchor]:
                    continue

                grid_map[anchor][side] = neighbor
                grid_map[neighbor][Image.opsosite_side[side]] = anchor

            queue.extend((n, tile_neigbors[n]) for n in neighbors)

        queue = deque(
            [
                (k, v, 0, 0)
                for k, v in grid_map.items()
                if "LEFT" not in v and "TOP" not in v
            ]
        )

        n = isqrt(len(tiles_map))
        grid = [[None] * n for _ in range(n)]

        while queue:
            tile, sides, r, c = queue.popleft()
            grid[r][c] = tile
            for side, neighbor in sides.items():
                if side == "TOP":
                    rr = r - 1
                    cc = c
                elif side == "BOTTOM":
                    rr = r + 1
                    cc = c
                elif side == "RIGHT":
                    rr = r
                    cc = c + 1
                elif side == "LEFT":
                    rr = r
                    cc = c - 1

                if grid[rr][cc] is None:
                    queue.append((neighbor, grid_map[neighbor], rr, cc))
        return grid

    def _get_neigbors(self, s) -> tuple[dict[int, Tile], dict[int, list[int]]]:
        tiles = [t.splitlines() for t in re.split(r"\n\s*\n", s.strip())]

        side_to_tiles = defaultdict(list)
        tiles_map = {}

        for tile in tiles:
            tile = Tile(tile)

            nr = tile.nr
            tiles_map[nr] = tile

            top, left, bottom, right = tile.top, tile.left, tile.bottom, tile.right

            side_to_tiles[top].append(nr)
            side_to_tiles[left].append(nr)
            side_to_tiles[bottom].append(nr)
            side_to_tiles[right].append(nr)

            side_to_tiles[top[::-1]].append(nr)
            side_to_tiles[left[::-1]].append(nr)
            side_to_tiles[bottom[::-1]].append(nr)
            side_to_tiles[right[::-1]].append(nr)

        possible_pairs = {
            tuple(sorted(p)) for p in side_to_tiles.values() if len(p) == 2
        }

        tile_neigbors = defaultdict(list)
        for t1, t2 in possible_pairs:
            tile_neigbors[t1].append(t2)
            tile_neigbors[t2].append(t1)

        return tiles_map, tile_neigbors

    def __repr__(self) -> str:
        return tabulate([[f"{c.nr}" for c in l] for l in self.grid])

    @classmethod
    def count_hashes_after_matching(cls, s: str) -> int:
        pattern = "                  # \n#    ##    ##    ###\n #  #  #  #  #  #   "
        pattern = [list(l) for l in pattern.splitlines()]
        p_rows, p_cols = len(pattern), len(pattern[0])

        image = Image(s)
        img = image.make_image()
        i_rows, i_cols = len(img), len(img[0])

        hashes_in_img = sum(
            1 for r, c in product(range(i_rows), range(i_cols)) if img[r][c] == "#"
        )
        hashes_in_pat = sum(
            1 for r, c in product(range(p_rows), range(p_cols)) if pattern[r][c] == "#"
        )

        for i in range(4):
            if i > 0:
                image.rotate_90_deg()
            count = image.count_pattern_match(pattern)
            if count > 0:
                return hashes_in_img - count * hashes_in_pat

        image.rotate_90_deg()
        image.flip_vertictal()

        for i in range(4):
            if i > 0:
                image.rotate_90_deg()
            count = image.count_pattern_match(pattern)
            if count > 0:
                return hashes_in_img - count * hashes_in_pat


Image.count_hashes_after_matching(example)

273

In [79]:
print(f"Part II: {Image.count_hashes_after_matching(puzzle)}")

Part II: 2009


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2009</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
